In [1]:
%run prepareStats.py

Last GP in the database: the 2022 Italian Grand Prix


In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
results = results[~((results["year"] < 1961) & (results["location"] == "Indianapolis"))]

In [4]:
results = results[results["laps"] > 0]

In [5]:
allDrivers = results["name"].drop_duplicates().to_list()

In [6]:
allDriversOnPodium = podiums["name"].drop_duplicates().to_list()

In [7]:
results["raced"] = results["raceId"].map(results.groupby("raceId")["name"].agg(list))

In [8]:
podiums["sharedPodium"] = podiums["raceId"].map(podiums.groupby("raceId")["name"].agg(list))

C:\Users\micha\AppData\Local\Temp\ipykernel_1368\3814610795.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  podiums["sharedPodium"] = podiums["raceId"].map(podiums.groupby("raceId")["name"].agg(list))


In [9]:
decidingraces = pd.read_csv(os.path.join("dataHandcollected", "titles.csv"))
decidingraces["date"] = pd.to_datetime(decidingraces["date"], format="%Y-%m-%d")
decidingraces["decidingRace"] = True
results = results.merge(decidingraces, on = ["date"], how = "left")

In [10]:
champions = results["champion"].drop_duplicates().to_list()

In [11]:
dateOfDebut = pd.Series(results.groupby(["driverId"])["date"].min(), name="dateOfDebut")
dateOfFarewell = pd.Series(results.groupby(["driverId"])["date"].max(), name="dateOfFarewell")
results = results.merge(dateOfDebut, on = ["driverId"], how = "left")
results = results.merge(dateOfFarewell, on = ["driverId"], how = "left")

## How many other World Champions has each World Champion competed against?

In [12]:
championAdversaries = {}
for champion in champions:
    championsRacedwith = []
    racedwith = results[results["name"] == champion]
    racedwith = racedwith["raced"].drop_duplicates().tolist()
    racedwith = [item for sublist in racedwith for item in sublist]
    racedwith = list(set(racedwith))
    for x in racedwith:
        if x in champions:
            championsRacedwith.append(x)
    for x in championsRacedwith:
        if x == champion:
            championsRacedwith.remove(x)
    racedNames = championsRacedwith
    racedCount = len(championsRacedwith)
    championAdversaries[champion] = [racedCount, ", ".join(racedNames)]
championAdversaries = pd.DataFrame.from_dict(championAdversaries, orient="index", columns=["count", "champions"])

championAdversaries.sort_values(by="count",ascending=False).head(10)

## How many drivers has each driver shared podiums with?

Technically speaking, some races had different drivers on podium than in the top-3 of the official results: DSQs etc. But let's find out anyway.

In [13]:
sharedPodiums = {}
sharedPodiumsCount = {}
for driver in allDriversOnPodium:
    sharedwith = podiums[podiums["name"] == driver]
    sharedwith = sharedwith["sharedPodium"].drop_duplicates().tolist()
    sharedwith = [item for sublist in sharedwith for item in sublist]
    sharedwith = list(set(sharedwith))
    for x in sharedwith:
        if x == driver:
            sharedwith.remove(x)
    sharedPodiums[driver] = sharedwith
    sharedPodiumsCount[driver] = [len(sharedwith), ", ".join(sharedwith)]
sharedPodiumsCount = pd.DataFrame.from_dict(sharedPodiumsCount, orient="index", columns=["count", "names"])    

In [14]:
sharedPodiumsCount.sort_values(by="count",ascending=False).head(10)

,count,names
Alain Prost,31,"Ayrton Senna, Stefano Modena, Carlos Reutemann, Eddie Cheever, Stefan Johansson, Martin Brundle, Patrick Tambay, Didier Pironi, René Arnoux, Bruno Giacomelli, Alan Jones, John Watson, Gerhard Berger, Damon Hill, Thierry Boutsen, Michele Alboreto, Keke Rosberg, Maurício Gugelmin, Alessandro Nannini, Nelson Piquet, Michael Schumacher, Mika Häkkinen, Ivan Capelli, Elio de Angelis, Mark Blundell, Niki Lauda, Derek Warwick, Riccardo Patrese, Jacques Laffite, Nigel Mansell, Andrea de Cesaris"
Michael Schumacher,31,"Ayrton Senna, Giancarlo Fisichella, Felipe Massa, Nick Heidfeld, Martin Brundle, Kimi Räikkönen, Jean Alesi, Robert Kubica, Fernando Alonso, Gerhard Berger, Alexander Wurz, Takuma Sato, Nigel Mansell, Damon Hill, Jarno Trulli, Jenson Button, David Coulthard, Tiago Monteiro, Nicola Larini, Heinz-Harald Frentzen, Eddie Irvine, Jacques Villeneuve, Juan Pablo Montoya, Mika Häkkinen, Mark Blundell, Alain Prost, Jos Verstappen, Riccardo Patrese, Ralf Schumacher, Rubens Barrichello, Johnny Herbert"
Niki Lauda,28,"Ayrton Senna, Carlos Reutemann, Emerson Fittipaldi, Patrick Tambay, Gunnar Nilsson, Didier Pironi, Jochen Mass, René Arnoux, Alan Jones, John Watson, Jody Scheckter, James Hunt, Clay Regazzoni, Michele Alboreto, Hans-Joachim Stuck, Keke Rosberg, Patrick Depailler, Nelson Piquet, Ronnie Peterson, Mario Andretti, Denny Hulme, Alain Prost, Derek Warwick, Riccardo Patrese, Jacques Laffite, Carlos Pace, Tom Pryce, Nigel Mansell"
Lewis Hamilton,27,"Felipe Massa, Heikki Kovalainen, Nick Heidfeld, Valtteri Bottas, Kimi Räikkönen, Charles Leclerc, Robert Kubica, Fernando Alonso, Daniel Ricciardo, Max Verstappen, Alexander Albon, Alexander Wurz, Timo Glock, Jarno Trulli, Jenson Button, Mark Webber, Esteban Ocon, Vitaly Petrov, Lando Norris, George Russell, Romain Grosjean, Nelson Piquet Jr., Sebastian Vettel, Rubens Barrichello, Carlos Sainz, Nico Rosberg, Sergio Pérez"
Juan Fangio,27,"Masten Gregory, Wolfgang von Trips, Louis Chiron, Alfonso de Portago, Piero Taruffi, Luigi Fagioli, Jean Behra, Carlos Menditeguy, Luigi Musso, Nino Farina, Maurice Trintignant, Karl Kling, Umberto Maglioli, Ron Flockhart, Peter Whitehead, Peter Collins, Eugenio Castellotti, Luigi Villoresi, Alberto Ascari, José Froilán González, Louis Rosier, Harry Schell, Hans Herrmann, Stirling Moss, Tony Brooks, Robert Manzon, Mike Hawthorn"
Carlos Reutemann,26,"Emerson Fittipaldi, Didier Pironi, Jochen Mass, René Arnoux, Alan Jones, John Watson, Jody Scheckter, James Hunt, Clay Regazzoni, Jacky Ickx, Gilles Villeneuve, François Cevert, Patrick Depailler, Nelson Piquet, Jean-Pierre Beltoise, Ronnie Peterson, Mario Andretti, Niki Lauda, Jean-Pierre Jabouille, Denny Hulme, Alain Prost, Riccardo Patrese, Jacques Laffite, Carlos Pace, Mike Hailwood, Nigel Mansell"
Kimi Räikkönen,25,"Giancarlo Fisichella, Felipe Massa, Lewis Hamilton, Heikki Kovalainen, Nick Heidfeld, Valtteri Bottas, Robert Kubica, Fernando Alonso, Nico Rosberg, Daniel Ricciardo, Max Verstappen, Timo Glock, Jarno Trulli, Jenson Button, David Coulthard, Heinz-Harald Frentzen, Michael Schumacher, Juan Pablo Montoya, Romain Grosjean, Sebastian Vettel, Ralf Schumacher, Rubens Barrichello, Pastor Maldonado, Mark Webber, Sergio Pérez"
Nelson Piquet,25,"Ayrton Senna, Stefano Modena, Carlos Reutemann, Eddie Cheever, Emerson Fittipaldi, Stefan Johansson, Patrick Tambay, Didier Pironi, René Arnoux, Teo Fabi, Alan Jones, John Watson, Gerhard Berger, Aguri Suzuki, Roberto Moreno, Thierry Boutsen, Michele Alboreto, Keke Rosberg, Elio de Angelis, Niki Lauda, Alain Prost, Riccardo Patrese, Jacques Laffite, Nigel Mansell, Andrea de Cesaris"
Ayrton Senna,24,"Stefano Modena, Stefan Johansson, Martin Brundle, Patrick Tambay, Jean Alesi, René Arnoux, Gerhard Berger, Damon Hill, Thierry Boutsen, Michele Alboreto, Keke Rosberg, Jyrki Järvilehto, Alessandro Nannini, Nelson Piquet, Mika Häkkinen, Michael Schumacher, Ivan Capelli, Mark Blundell, Niki Lauda, Alain Prost, Derek Warwick, Riccardo 

## The Wormholes

What's the longest span between the first race with a driver who would compete with driver X and the last race with a driver who had competed against driver X?

In [15]:
adversaries = {}
for driver in allDrivers:
    earliest = results["dateOfDebut"].max()
    latest = results["dateOfFarewell"].min()
    racedwith = results[results["name"] == driver]
    dateOfDebut = racedwith["dateOfDebut"].min()
    dateOfFarewell = racedwith["dateOfFarewell"].max()
    racedwith = racedwith["raced"].drop_duplicates().tolist()
    racedwith = [item for sublist in racedwith for item in sublist]
    racedwith = list(set(racedwith))
    for x in racedwith:
        if x == driver:
            racedwith.remove(x)
        else:
            earliestDebut = results[results["name"] == x]["dateOfDebut"].min()
            latestFarewell = results[results["name"] == x]["dateOfFarewell"].max()
            if earliestDebut < earliest:
                earliest = earliestDebut
            if latestFarewell > latest:
                latest = latestFarewell
    racedNames = racedwith
    racedCount = len(racedwith)
    adversaries[driver] = [dateOfDebut, dateOfFarewell, racedCount, ", ".join(racedNames), earliest, latest]
adversaries = pd.DataFrame.from_dict(adversaries, orient="index", columns=["first GP", "last GP", "count", "names", "earliest", "latest"])
adversaries.index.name = "name"
adversaries["span"] = adversaries["latest"] - adversaries["earliest"]

In [16]:
adversaries[["first GP","last GP","count","earliest","latest","span"]].sort_values(by="span",ascending=False).head(50)

,first GP,last GP,count,earliest,latest,span
name,,,,,,
Riccardo Patrese,1977-05-22,1993-11-07,147,1967-08-06,2012-11-25,16548 days
Jan Lammers,1979-01-21,1992-11-08,70,1967-08-06,2012-11-25,16548 days
Rubens Barrichello,1993-03-14,2011-11-27,136,1977-05-22,2022-09-11,16548 days
Michael Schumacher,1991-09-08,2012-11-25,143,1977-05-22,2022-09-11,16548 days
Eddie Irvine,1993-10-24,2002-10-13,86,1977-05-22,2022-09-11,16548 days
Mika Häkkinen,1991-03-10,2001-10-14,97,1977-05-22,2022-09-11,16548 days
Jean Alesi,1989-07-09,2001-10-14,108,1977-05-22,2022-09-11,16548 days
Nelson Piquet,1978-07-30,1991-11-03,119,1967-08-06,2012-11-25,16548 days
Luca Badoer,1993-03-14,2009-08-30,77,1977-05-22,2022-09-11,16548 days


In [17]:
adversaries[["count"]].sort_values(by="count",ascending=False).head(10)

,count
name,
Graham Hill,236
Maurice Trintignant,235
Jack Brabham,214
Jo Bonnier,206
Stirling Moss,195
Roy Salvadori,187
Harry Schell,176
Bruce McLaren,173
John Surtees,168


In [18]:
len(allDrivers)/236

2.8389830508474576

More than one third of F1 drivers have competed against Graham Hill.

In [19]:
lastGPdriversAdversaries = adversaries[adversaries.index.isin(lastGPdrivers)]

In [20]:
lastGPdriversAdversaries[["count"]].sort_values(by="count",ascending=False).head(10)

,count
name,
Fernando Alonso,113
Lewis Hamilton,85
Sebastian Vettel,85
Sergio Pérez,70
Daniel Ricciardo,70
Valtteri Bottas,56
Kevin Magnussen,49
Carlos Sainz,46
Max Verstappen,45


## Fangio Number

Inspired by the [Erdős number](https://en.wikipedia.org/wiki/Erd%C5%91s_number): J-M Fangio's own Fangio number was 0. Each driver who has raced against JMF at least for one lap has Fangio number 1, each driver who has raced against anyone with Fangio number 1 has Fangio number 2 etc.

In [21]:
fangio0 = ["Juan Fangio"]

In [22]:
fangio1, fangio2, fangio3, fangio4, fangio5, fangio6, fangio7, alreadycounted = [], [], [], [], [], [], [], []
for driver in allDrivers:
    racedwith = results[results["name"] == driver]
    racedwith = racedwith["raced"].drop_duplicates().tolist()
    racedwith = [item for sublist in racedwith for item in sublist]
    racedwith = list(set(racedwith))
    for x in racedwith:
        if x in fangio0:
            if driver not in fangio0:
                fangio1.append(driver)
                alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in fangio1:
                if driver not in fangio0:
                    if driver not in fangio1:
                        fangio2.append(driver)
                        alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in fangio2:
                if driver not in fangio0:
                    if driver not in fangio1:
                        if driver not in fangio2:
                            fangio3.append(driver)
                            alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in fangio3:
                if driver not in fangio0:
                    if driver not in fangio1:
                        if driver not in fangio2:
                            if driver not in fangio3:
                                fangio4.append(driver)
                                alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in fangio4:
                if driver not in fangio0:
                    if driver not in fangio1:
                        if driver not in fangio2:
                            if driver not in fangio3:
                                if driver not in fangio4:
                                    fangio5.append(driver)
                                    alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in fangio5:
                if driver not in fangio0:
                    if driver not in fangio1:
                        if driver not in fangio2:
                            if driver not in fangio3:
                                if driver not in fangio4:
                                    if driver not in fangio5:
                                        fangio6.append(driver)
                                        alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in fangio6:
                if driver not in fangio0:
                    if driver not in fangio1:
                        if driver not in fangio2:
                            if driver not in fangio3:
                                if driver not in fangio4:
                                    if driver not in fangio5:
                                        if driver not in fangio6:
                                            fangio7.append(driver)
                                            alreadycounted.append(driver)

In [23]:
results.loc[results.name.isin(fangio0),"fangioNumber"]=0
results.loc[results.name.isin(fangio1),"fangioNumber"]=1
results.loc[results.name.isin(fangio2),"fangioNumber"]=2
results.loc[results.name.isin(fangio3),"fangioNumber"]=3
results.loc[results.name.isin(fangio4),"fangioNumber"]=4
results.loc[results.name.isin(fangio5),"fangioNumber"]=5
results.loc[results.name.isin(fangio6),"fangioNumber"]=6

In [24]:
fangioNumbers = {}
for x in range(0,7):
    fangioResults = results[results["fangioNumber"] == x]
    fangioResults.sort_values(by="date")
    fangioGeneration = fangioResults["name"].drop_duplicates().to_list()
    earliest = fangioResults["date"].min()
    earliestDrivers = fangioResults[fangioResults["date"] == earliest]
    earliestDrivers = earliestDrivers["name"].to_list()
    latest = fangioResults["date"].max()
    latestDrivers = fangioResults[fangioResults["date"] == latest]
    latestDrivers = latestDrivers["name"].to_list()
    drivers = len(fangioGeneration)
    fangioChamps = []
    for champ in champions:
        if champ in fangioGeneration:
            fangioChamps.append(champ)
    fanChamps = len(fangioChamps)
    fangioNumbers[x] = [drivers, earliest, ", ".join(earliestDrivers), latest, ", ".join(latestDrivers), fanChamps, ", ".join(fangioChamps)]
fangioNumbers = pd.DataFrame.from_dict(fangioNumbers, orient="index", columns=["drivers", "earliest", "earliest drivers", "latest", "latest drivers", "no_champs", "champions"])

In [25]:
fangioNumbers[["earliest","earliest drivers","latest","latest drivers"]]

,earliest,earliest drivers,latest,latest drivers
0,1950-05-13,Juan Fangio,1958-07-06,Juan Fangio
1,1950-05-13,"Louis Chiron, Bob Gerard, Louis Rosier, Toulo de Graffenried, Nino Farina, Johnny Claes, Peter Walker, Tony Rolt, Prince Bira, Reg Parnell, Yves Cabantous, Philippe Étancelin, David Murray, Joe Kelly, Brian Shawe Taylor, David Hampshire, Geoff Crossley, Luigi Fagioli, Cuth Harrison, Joe Fry, Eugène Martin, Leslie Johnson",1975-01-26,Graham Hill
2,1952-05-18,Toni Ulmen,1986-07-06,Jacques Laffite
3,1973-02-11,Luiz Bueno,1997-10-26,Gerhard Berger
4,1986-09-07,Alex Caffi,2012-11-25,Michael Schumacher
5,1997-06-15,Alexander Wurz,2022-09-11,"Lewis Hamilton, Fernando Alonso, Sebastian Vettel, Sergio Pérez, Daniel Ricciardo"
6,2013-03-17,"Max Chilton, Esteban Gutiérrez, Valtteri Bottas, Giedo van der Garde, Jules Bianchi",2022-09-11,"Valtteri Bottas, Kevin Magnussen, Max Verstappen, Carlos Sainz, Esteban Ocon, Lance Stroll, Pierre Gasly, Charles Leclerc, Lando Norris, George Russell, Nicholas Latifi, Yuki Tsunoda, Mick Schumacher, Guanyu Zhou, Nyck de Vries"


In [26]:
fangioNumbers[["no_champs", "champions"]]

,no_champs,champions
0,1,Juan Fangio
1,6,"Graham Hill, Mike Hawthorn, Phil Hill, Jack Brabham, Alberto Ascari, Nino Farina"
2,10,"Niki Lauda, Mario Andretti, Jody Scheckter, James Hunt, Emerson Fittipaldi, Jackie Stewart, Jochen Rindt, Denny Hulme, Jim Clark, John Surtees"
3,6,"Alain Prost, Ayrton Senna, Nigel Mansell, Nelson Piquet, Keke Rosberg, Alan Jones"
4,4,"Michael Schumacher, Mika Häkkinen, Jacques Villeneuve, Damon Hill"
5,6,"Lewis Hamilton, Jenson Button, Sebastian Vettel, Kimi Räikkönen, Nico Rosberg, Fernando Alonso"
6,1,Max Verstappen


In [27]:
fangioNumbers["drivers"].sum()

670

## Farina Number

It could also make sense to categorise all the drivers by their Farina Number – Giuseppe Farina won the first GP ever and was the first World Champion.

In [28]:
farina0 = ["Nino Farina"]

In [29]:
farina1, farina2, farina3, farina4, farina5, farina6, farina7, alreadycounted = [], [], [], [], [], [], [], []
for driver in allDrivers:
    racedwith = results[results["name"] == driver]
    racedwith = racedwith["raced"].drop_duplicates().tolist()
    racedwith = [item for sublist in racedwith for item in sublist]
    racedwith = list(set(racedwith))
    for x in racedwith:
        if x in farina0:
            if driver not in farina0:
                farina1.append(driver)
                alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in farina1:
                if driver not in farina0:
                    if driver not in farina1:
                        farina2.append(driver)
                        alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in farina2:
                if driver not in farina0:
                    if driver not in farina1:
                        if driver not in farina2:
                            farina3.append(driver)
                            alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in farina3:
                if driver not in farina0:
                    if driver not in farina1:
                        if driver not in farina2:
                            if driver not in farina3:
                                farina4.append(driver)
                                alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in farina4:
                if driver not in farina0:
                    if driver not in farina1:
                        if driver not in farina2:
                            if driver not in farina3:
                                if driver not in farina4:
                                    farina5.append(driver)
                                    alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in farina5:
                if driver not in farina0:
                    if driver not in farina1:
                        if driver not in farina2:
                            if driver not in farina3:
                                if driver not in farina4:
                                    if driver not in farina5:
                                        farina6.append(driver)
                                        alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in farina6:
                if driver not in farina0:
                    if driver not in farina1:
                        if driver not in farina2:
                            if driver not in farina3:
                                if driver not in farina4:
                                    if driver not in farina5:
                                        if driver not in farina6:
                                            farina7.append(driver)
                                            alreadycounted.append(driver)

In [30]:
results.loc[results.name.isin(farina0),"farinaNumber"]=0
results.loc[results.name.isin(farina1),"farinaNumber"]=1
results.loc[results.name.isin(farina2),"farinaNumber"]=2
results.loc[results.name.isin(farina3),"farinaNumber"]=3
results.loc[results.name.isin(farina4),"farinaNumber"]=4
results.loc[results.name.isin(farina5),"farinaNumber"]=5
results.loc[results.name.isin(farina6),"farinaNumber"]=6
results.loc[results.name.isin(farina7),"farinaNumber"]=7

In [31]:
farinaNumbers = {}
for x in range(0,8):
    farinaResults = results[results["farinaNumber"] == x]
    farinaResults.sort_values(by="date")
    farinaGeneration = farinaResults["name"].drop_duplicates().to_list()
    earliest = farinaResults["date"].min()
    earliestDrivers = farinaResults[farinaResults["date"] == earliest]
    earliestDrivers = earliestDrivers["name"].to_list()
    latest = farinaResults["date"].max()
    latestDrivers = farinaResults[farinaResults["date"] == latest]
    latestDrivers = latestDrivers["name"].to_list()
    drivers = len(farinaGeneration)
    farinaChamps = []
    for champ in champions:
        if champ in farinaGeneration:
            farinaChamps.append(champ)
    fanChamps = len(farinaChamps)
    farinaNumbers[x] = [drivers, earliest, ", ".join(earliestDrivers), latest, ", ".join(latestDrivers), fanChamps, ", ".join(farinaChamps)]
farinaNumbers = pd.DataFrame.from_dict(farinaNumbers, orient="index", columns=["drivers", "earliest", "earliest drivers", "latest", "latest drivers", "no_champs", "champions"])

In [32]:
farinaNumbers[["drivers", "earliest","earliest drivers","latest","latest drivers"]]

,drivers,earliest,earliest drivers,latest,latest drivers
0,1,1950-05-13,Nino Farina,1955-06-05,Nino Farina
1,137,1950-05-13,"Juan Fangio, Louis Chiron, Bob Gerard, Louis Rosier, Toulo de Graffenried, Johnny Claes, Peter Walker, Tony Rolt, Prince Bira, Reg Parnell, Yves Cabantous, Philippe Étancelin, David Murray, Joe Kelly, Brian Shawe Taylor, David Hampshire, Geoff Crossley, Luigi Fagioli, Cuth Harrison, Joe Fry, Eugène Martin, Leslie Johnson",1964-09-06,Maurice Trintignant
2,132,1954-07-17,"Ron Flockhart, Horace Gould, Leslie Marr, Don Beauman, Leslie Thorne, Bill Whitehouse, John Riseley-Prichard",1976-07-18,Chris Amon
3,133,1960-06-19,Lance Reventlow,1986-10-26,Alan Jones
4,85,1975-03-01,Guy Tunmer,1997-10-26,Gerhard Berger
5,77,1987-04-12,"Satoru Nakajima, Adrián Campos, Pascal Fabre",2012-11-25,Michael Schumacher
6,68,1997-06-15,Alexander Wurz,2022-09-11,"Lewis Hamilton, Fernando Alonso, Sebastian Vettel, Sergio Pérez, Daniel Ricciardo"
7,37,2013-03-17,"Max Chilton, Esteban Gutiérrez, Valtteri Bottas, Giedo van der Garde, Jules Bianchi",2022-09-11,"Valtteri Bottas, Kevin Magnussen, Max Verstappen, Carlos Sainz, Esteban Ocon, Lance Stroll, Pierre Gasly, Charles Leclerc, Lando Norris, George Russell, Nicholas Latifi, Yuki Tsunoda, Mick Schumacher, Guanyu Zhou, Nyck de Vries"


In [33]:
farinaNumbers[["no_champs", "champions"]]

,no_champs,champions
0,1,Nino Farina
1,3,"Mike Hawthorn, Juan Fangio, Alberto Ascari"
2,5,"Graham Hill, Jim Clark, John Surtees, Phil Hill, Jack Brabham"
3,9,"Niki Lauda, Alan Jones, Mario Andretti, Jody Scheckter, James Hunt, Emerson Fittipaldi, Jackie Stewart, Jochen Rindt, Denny Hulme"
4,5,"Alain Prost, Ayrton Senna, Nigel Mansell, Nelson Piquet, Keke Rosberg"
5,4,"Michael Schumacher, Mika Häkkinen, Jacques Villeneuve, Damon Hill"
6,6,"Lewis Hamilton, Jenson Button, Sebastian Vettel, Kimi Räikkönen, Nico Rosberg, Fernando Alonso"
7,1,Max Verstappen
